In [1]:
# %pip install python-dotenv

In [6]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access an environment variable
endpoint = os.getenv("endpoint")
key = os.getenv("key")

# print(f"Endpoint: {endpoint}")
# print(f"Key: {key}")
print("Environment variables loaded successfully.")

Environment variables loaded successfully.


In [7]:
import requests

analyzerId = "document_analyzer_01"  # Replace with your actual analyzer ID
url = f"{endpoint}/contentunderstanding/analyzers/{analyzerId}?api-version=2024-12-01-preview"
headers = {
    "Ocp-Apim-Subscription-Key": key,
    "Content-Type": "application/json"
}

with open('document_request_body.json', 'r') as file:
    data = file.read()

response = requests.put(url, headers=headers, data=data)

print(response.status_code)
print(response.json())

409
{'error': {'code': 'Conflict', 'message': 'The request could not be completed due to a conflict with the current state of the target resource.', 'innererror': {'code': 'ModelExists', 'message': 'A model with the provided name already exists.'}}}


In [8]:
# print(response.headers)
operation_location = response.headers.get('Operation-Location')
# print(f"Operation-Location: {operation_location}")

In [9]:
operationId = "your_operation_id"  # Replace with your actual operation ID
url = operation_location

response = requests.get(url, headers=headers)

print(response.status_code)
print(response.json())

MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?

In [10]:
file_url = "https://github.com/Azure-Samples/cognitive-services-REST-api-samples/raw/master/curl/form-recognizer/rest-api/invoice.pdf"

analyze_url = f"{endpoint}/contentunderstanding/analyzers/{analyzerId}:analyze?api-version=2024-12-01-preview"
payload = {"url": file_url}

response = requests.post(analyze_url, headers=headers, json=payload)

print(response.status_code)
print(response.json())



202
{'id': '30f0dd97-ec7f-4caa-a385-0f2c1a528c07', 'status': 'Running', 'result': {'analyzerId': 'document_analyzer_01', 'apiVersion': '2024-12-01-preview', 'createdAt': '2025-05-06T00:12:30Z', 'warnings': [], 'contents': []}}


In [11]:
operation_location = response.headers.get('Operation-Location')
# print(f"Operation-Location: {operation_location}")

In [12]:
import json

# resultId = "c62115c9-090e-4fa2-b326-e0eac6f0e0be"  # Replace with your actual result ID
results_url = operation_location

response = requests.get(results_url, headers=headers)

print(response.status_code)

json_response = response.json()
print(json.dumps(json_response, indent=4))

200
{
    "id": "30f0dd97-ec7f-4caa-a385-0f2c1a528c07",
    "status": "Succeeded",
    "result": {
        "analyzerId": "document_analyzer_01",
        "apiVersion": "2024-12-01-preview",
        "createdAt": "2025-05-06T00:12:30Z",
        "warnings": [],
        "contents": [
            {
                "markdown": "CONTOSO LTD.\n\n\n# INVOICE\n\nContoso Headquarters\n123 456th St\nNew York, NY, 10001\n\nINVOICE: INV-100\n\nINVOICE DATE: 11/15/2019\n\nDUE DATE: 12/15/2019\n\nCUSTOMER NAME: MICROSOFT CORPORATION\n\nSERVICE PERIOD: 10/14/2019 - 11/14/2019\n\nCUSTOMER ID: CID-12345\n\nMicrosoft Corp\n123 Other St,\nRedmond WA, 98052\n\nBILL TO:\n\nMicrosoft Finance\n\n123 Bill St,\n\nRedmond WA, 98052\n\nSHIP TO:\n\nMicrosoft Delivery\n\n123 Ship St,\n\nRedmond WA, 98052\n\nSERVICE ADDRESS:\nMicrosoft Services\n123 Service St,\nRedmond WA, 98052\n\n\n<table>\n<tr>\n<th>SALESPERSON</th>\n<th>P.O. NUMBER</th>\n<th>REQUISITIONER</th>\n<th>SHIPPED VIA</th>\n<th>F.O.B. POINT</th>\n<th>TER

In [13]:
from pydantic import BaseModel
from typing import List, Optional, Dict, Any

class Item(BaseModel):
    Description: Optional[str]
    Amount: Optional[float]

class FieldSchema(BaseModel):
    VendorName: Optional[str]
    Items: Optional[List[Item]]

class Result(BaseModel):
    analyzerId: str
    apiVersion: str
    createdAt: str
    warnings: Optional[List[Any]]
    contents: Optional[List[Any]]

class JsonResponse(BaseModel):
    id: str
    status: str
    result: Result

# Convert json_response to Pydantic object
pydantic_response = JsonResponse(**json_response)
print(pydantic_response)

id='30f0dd97-ec7f-4caa-a385-0f2c1a528c07' status='Succeeded' result=Result(analyzerId='document_analyzer_01', apiVersion='2024-12-01-preview', createdAt='2025-05-06T00:12:30Z', warnings=[], contents=[{'markdown': 'CONTOSO LTD.\n\n\n# INVOICE\n\nContoso Headquarters\n123 456th St\nNew York, NY, 10001\n\nINVOICE: INV-100\n\nINVOICE DATE: 11/15/2019\n\nDUE DATE: 12/15/2019\n\nCUSTOMER NAME: MICROSOFT CORPORATION\n\nSERVICE PERIOD: 10/14/2019 - 11/14/2019\n\nCUSTOMER ID: CID-12345\n\nMicrosoft Corp\n123 Other St,\nRedmond WA, 98052\n\nBILL TO:\n\nMicrosoft Finance\n\n123 Bill St,\n\nRedmond WA, 98052\n\nSHIP TO:\n\nMicrosoft Delivery\n\n123 Ship St,\n\nRedmond WA, 98052\n\nSERVICE ADDRESS:\nMicrosoft Services\n123 Service St,\nRedmond WA, 98052\n\n\n<table>\n<tr>\n<th>SALESPERSON</th>\n<th>P.O. NUMBER</th>\n<th>REQUISITIONER</th>\n<th>SHIPPED VIA</th>\n<th>F.O.B. POINT</th>\n<th>TERMS</th>\n</tr>\n<tr>\n<td></td>\n<td>PO-3333</td>\n<td></td>\n<td></td>\n<td></td>\n<td></td>\n</tr>\n</table

In [14]:
pydantic_response.result.contents[0].keys()


dict_keys(['markdown', 'fields', 'kind', 'startPageNumber', 'endPageNumber', 'unit', 'pages', 'paragraphs', 'sections', 'tables'])

In [15]:
pydantic_response.result.contents[0]['fields']

{'VendorName': {'type': 'string',
  'valueString': 'CONTOSO LTD.',
  'spans': [{'offset': 0, 'length': 12}],
  'confidence': 0.92,
  'source': 'D(1,0.5729,0.6582,2.3353,0.6582,2.3353,0.8957,0.5729,0.8957)'},
 'Items': {'type': 'array',
  'valueArray': [{'type': 'object',
    'valueObject': {'Description': {'type': 'string',
      'valueString': 'Consulting Services',
      'spans': [{'offset': 909, 'length': 19}],
      'confidence': 0.973,
      'source': 'D(1,2.3264,5.673,3.6413,5.673,3.6413,5.8402,2.3264,5.8402)'},
     'Amount': {'type': 'number',
      'valueNumber': 60,
      'spans': [{'offset': 995, 'length': 6}],
      'confidence': 0.996,
      'source': 'D(1,7.4507,5.6684,7.9245,5.6684,7.9245,5.8323,7.4507,5.8323)'}}},
   {'type': 'object',
    'valueObject': {'Description': {'type': 'string',
      'valueString': 'Document Fee',
      'spans': [{'offset': 1054, 'length': 12}],
      'confidence': 0.99,
      'source': 'D(1,2.3255,5.9758,3.3258,5.9758,3.3258,6.1319,2.3255,6.